In [93]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt 
import pickle
from visualization_server import * 
import numpy as np
from scipy import fftpack
import tensorly as tl
from util import square_tensor_gen, TensorInfoBucket, RandomInfoBucket, eval_mse, eval_rerr
from sketch import Sketch
import time
from tensorly.decomposition import tucker
from sketch_recover import SketchTwoPassRecover
from sketch_recover import SketchOnePassRecover


data = nc.Dataset("data/b.e11.B20TRC5CNBDRD.f09_g16.104.cice.h.rain_ai_sh.192001-200512.nc")
rain_ai = data.variables['rain_ai'][:] 
pickle.dump( rain_ai, open("data/rain_ai.pickle", "wb" ) )
rain_ai = pickle.load( open( "data/rain_ai.pickle", "rb" ) ) 

Using mxnet backend.
/Users/apple/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/apple/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
Using numpy backend.


In [94]:
train = rain_ai.filled(rain_ai.mean())
train.shape

(1032, 76, 320)

In [139]:
def run_hosvd(X,rk,random_seed = 1):
    core, tucker_factors = tucker(X, ranks = [rk for _ in range(X.ndim)], init = 'svd')
    X_hat = tl.tucker_to_tensor(core, tucker_factors)
    rerr = eval_rerr(X,X_hat,X)
    return rerr

def run_twopass(X,rk,k,random_seed = 1): 
    dim = X.ndim
    sketch = Sketch(X, k, random_seed = random_seed)  
    sketchs,_  = sketch.get_sketchs()
    sketch_two_pass = SketchTwoPassRecover(X, sketchs, np.repeat(rk,dim))
    X_hat,_,_ = sketch_two_pass.recover()
    rerr = eval_rerr(X,X_hat,X)
    return rerr
    
def run_onepass(X,rk,k,s,random_seed = 1, store_phis = True):
    dim = X.ndim
    sketch = Sketch(X,k,random_seed = random_seed, s = s, store_phis = store_phis)
    sketchs, core_sketch = sketch.get_sketchs()
    sketch_one_pass = SketchOnePassRecover(sketchs,core_sketch,\
            TensorInfoBucket(X.shape,k,np.repeat\
                (rk,dim),s),RandomInfoBucket(random_seed = random_seed),\
            sketch.get_phis())
    X_hat, _,_ = sketch_one_pass.recover()
    rerr = eval_rerr(X,X_hat,X)
    return rerr

In [140]:
def simreal_name(k): 
    return "realdata_k"+k
    
def run_realdata(data = "rain_ai",t = 200,sim_runs = 1,random_seed = 1): 
    rain_ai = pickle.load( open( "data/"+data+".pickle", "rb" ) ) 
    train = rain_ai.filled(rain_ai.mean())
    # Fill in the missing value with the average value
    train = train[:,:,range(t)]
    k = int(np.ceil(1/2.1*min(train.shape)))
    rks = (np.ceil(np.arange(0.1,0.6,0.1)*k)).astype(int)
    print(k,rks)
    ho_svd_rerr = np.zeros(rks.size)
    two_pass_rerr = np.zeros(rks.size)
    one_pass_rerr = np.zeros(rks.size)
    for idrk, rk in enumerate(rks):               
        s = 2*k +1
        ho_svd_rerr[idrk] = run_hosvd(train,rk) 
        two_pass_rerr[idrk] = run_twopass(train,rk,k) 
        one_pass_rerr[idrk] = run_onepass(train,rk,k,s)
    return [ho_svd_rerr, two_pass_rerr, one_pass_rerr]

In [141]:
result = run_realdata()

37 [ 4  8 12 15 19]


Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.


In [142]:
result

[array([0.70594174, 0.60592282, 0.54697168, 0.51269895, 0.47556704]),
 array([0.72867953, 0.65183343, 0.61373024, 0.59423629, 0.57598528]),
 array([0.76824155, 0.74669489, 0.74827961, 0.75281235, 0.75680434])]